In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
from pathlib import Path
import pandas as pd
from tqdm import tqdm
import json
import numpy as np
import pickle
import cv2

import matplotlib.pyplot as plt
import seaborn as sns
from openslide import OpenSlide

In [4]:
from probreg import cpd
from probreg import callbacks
from probreg import transformation as tf

In [5]:
plt.rcParams.update({'font.size': 25})

patch_size = 1024

In [6]:
def add_help_fields(frame):
    
    frame["image_name_stem"] = [Path(image_name).stem for image_name in frame["image_name"]]
    
    frame["patient_id"] = [name.split("_")[2] for name in frame["image_name"]]

    frame["x1"] = [json.loads(vector.replace("\'","\""))['x1'] for vector in frame["vector"]]
    frame["y1"] = [json.loads(vector.replace("\'","\""))['y1'] for vector in frame["vector"]]

    frame["x2"] = [json.loads(vector.replace("\'","\""))['x2'] for vector in frame["vector"]]
    frame["y2"] = [json.loads(vector.replace("\'","\""))['y2'] for vector in frame["vector"]]

    frame["center_x"] = [x1 + ((x2-x1) / 2) for x1, x2 in zip(frame["x1"], frame["x2"])]
    frame["center_y"] = [y1 + ((y2-y1) / 2) for y1, y2 in zip(frame["y1"], frame["y2"])]
    
    frame["center"] = [[center_x, center_y] for center_x, center_y in zip(frame["center_x"], frame["center_y"])]

    frame["anno_width"] = [x2-x1 for x1, x2 in zip(frame["x1"], frame["x2"])]
    frame["anno_height"]= [y2-y1 for y1, y2 in zip(frame["y1"], frame["y2"])]
    
    return frame

In [7]:
gt_annotations = add_help_fields(pd.read_csv("GT.csv"))
gt_annotations.head()

,scanner,image_id,image_name,image_type,image_width,image_height,id,vector,unique_identifier,annotation_type,...,patient_id,x1,y1,x2,y2,center_x,center_y,center,anno_width,anno_height
0,2.0HT,10410,N2_CCMCT_380609B_1.ndpi,CCMCT,147456,107008,2755406,"{'x1': 20354, 'x2': 20408, 'y1': 16752, 'y2': ...",95183f6e-0a00-4de4-9c97-fedfec655a27,384,...,380609B,20354,16752,20408,16806,20381.0,16779.0,"[20381.0, 16779.0]",54,54
1,2.0HT,10410,N2_CCMCT_380609B_1.ndpi,CCMCT,147456,107008,2755407,"{'x1': 20256, 'x2': 20310, 'y1': 35448, 'y2': ...",cc3def2b-5631-4619-ab08-51f2a69c667d,385,...,380609B,20256,35448,20310,35502,20283.0,35475.0,"[20283.0, 35475.0]",54,54
2,2.0HT,10410,N2_CCMCT_380609B_1.ndpi,CCMCT,147456,107008,2755408,"{'x1': 20445, 'x2': 20499, 'y1': 54139, 'y2': ...",2d3de917-e55a-470f-8af2-10eb49b69687,386,...,380609B,20445,54139,20499,54193,20472.0,54166.0,"[20472.0, 54166.0]",54,54
3,2.0HT,10410,N2_CCMCT_380609B_1.ndpi,CCMCT,147456,107008,2755409,"{'x1': 26495, 'x2': 26549, 'y1': 71854, 'y2': ...",611f9d35-6cdb-42ef-bc38-2f44c9aaa257,387,...,380609B,26495,71854,26549,71908,26522.0,71881.0,"[26522.0, 71881.0]",54,54
4,2.0HT,10410,N2_CCMCT_380609B_1.ndpi,CCMCT,147456,107008,2755410,"{'x1': 35443, 'x2': 35497, 'y1': 90091, 'y2': ...",bf3ef0d3-5153-4693-96ba-96047cb3ae24,388,...,380609B,35443,90091,35497,90145,35470.0,90118.0,"[35470.0, 90118.0]",54,54


In [33]:
results, quality = [], []


for patient_id in tqdm(gt_annotations["patient_id"].unique()):
    
    patient_annos = gt_annotations[gt_annotations["patient_id"] == patient_id]
    
    source_annos = patient_annos[patient_annos["scanner"] == "Aperio"]
    source_anno = source_annos.iloc[0]
    
    for scanner in ['2.0HT', 'Axio', 'S210']:
        
        target_annos = patient_annos[patient_annos["scanner"] == scanner]
        target_anno = target_annos.iloc[0]
        
        # filter to match same number of points
        intersections = list(set(source_annos["annotation_type"]).intersection(target_annos["annotation_type"]))
        source_annos = source_annos[source_annos["annotation_type"].isin(intersections)]
        target_annos = target_annos[target_annos["annotation_type"].isin(intersections)]
        
        source = np.array([a for a in source_annos["center"]])
        target = np.array([a for a in target_annos["center"]])

        #tf_param, sigma2, q = cpd.registration_cpd(source, target, 'affine')
        homography, mask = cv2.findHomography(source, target) # , cv2.RANSAC
        tf_param = tf.AffineTransformation(homography[:2, :2], homography[:2, 2:].reshape(-1))
        
        
        mpp_x_scale = tf_param.b[0][0]
        mpp_y_scale = tf_param.b[1][1]
        
        
        target_scanner = target_anno.scanner 
        image_id = target_anno.image_id 
        image_name = target_anno.image_name 
        image_width, image_height = target_anno.image_width, target_anno.image_height
        
        image_type = "CCMCT" if "CCMCT" in image_name else "Cyto"
        
        quality.append([patient_id, source_anno.image_name, target_anno.image_name, 
                        image_type, tf_param.b, tf_param.t])
        
        for id, source_anno in source_annos.iterrows():          
            
            # trans center
            box_1 = [source_anno.center_x, source_anno.center_y]
            trans_box_1 = tf_param.transform([box_1])[0]
            
            # trans width and hight
            new_width, new_height = source_anno.anno_width * mpp_x_scale, source_anno.anno_height * mpp_y_scale
            
            trans_box = [trans_box_1[0], trans_box_1[1], new_width, new_height]
            
            new_x1 = int(trans_box[0] - trans_box[2] // 2)
            new_y1 = int(trans_box[1] - trans_box[3] // 2)
            new_x2 = int(trans_box[0] + trans_box[2] // 2)
            new_y2 = int(trans_box[1] + trans_box[3] // 2)
            
            vector = {'x1': new_x1, 'x2': new_x2, 'y1': new_y1, 'y2': new_y2}
            
            row = [target_scanner, image_id, image_name, image_type, image_width, image_height,
                    vector, source_anno.unique_identifier, source_anno.annotation_type, 
                   source_anno.type_name, "GT_Registration"]
            
            results.append(row)

quality = pd.DataFrame(quality, columns=["patient_id", "source", "target", "image_type", "b", "t"])

results = pd.DataFrame(results, columns=["scanner", "image_id", "image_name", "image_type", "image_width", 
                                         "image_height", "vector", "unique_identifier", "annotation_type", "type_name", "method"])
results.head()

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 55.69it/s]


,scanner,image_id,image_name,image_type,image_width,image_height,vector,unique_identifier,annotation_type,type_name,method
0,2.0HT,10410,N2_CCMCT_380609B_1.ndpi,CCMCT,147456,107008,"{'x1': 20354, 'x2': 20408, 'y1': 16756, 'y2': ...",95183f6e-0a00-4de4-9c97-fedfec655a27,384,L0,GT_Registration
1,2.0HT,10410,N2_CCMCT_380609B_1.ndpi,CCMCT,147456,107008,"{'x1': 20253, 'x2': 20307, 'y1': 35450, 'y2': ...",cc3def2b-5631-4619-ab08-51f2a69c667d,385,L1,GT_Registration
2,2.0HT,10410,N2_CCMCT_380609B_1.ndpi,CCMCT,147456,107008,"{'x1': 20443, 'x2': 20497, 'y1': 54138, 'y2': ...",2d3de917-e55a-470f-8af2-10eb49b69687,386,L2,GT_Registration
3,2.0HT,10410,N2_CCMCT_380609B_1.ndpi,CCMCT,147456,107008,"{'x1': 26496, 'x2': 26550, 'y1': 71853, 'y2': ...",611f9d35-6cdb-42ef-bc38-2f44c9aaa257,387,L3,GT_Registration
4,2.0HT,10410,N2_CCMCT_380609B_1.ndpi,CCMCT,147456,107008,"{'x1': 35437, 'x2': 35491, 'y1': 90084, 'y2': ...",bf3ef0d3-5153-4693-96ba-96047cb3ae24,388,L4,GT_Registration


In [34]:
results.to_csv("GT_Registration_Home.csv", index=False)

In [32]:
quality

,patient_id,source,target,image_type,b,t
0,380609B,A_CCMCT_380609B_1.svs,N2_CCMCT_380609B_1.ndpi,CCMCT,"[[1.1184819360330307, 0.003107663856129997], [...","[3044.051138744005, 3194.1503661920556]"
1,380609B,A_CCMCT_380609B_1.svs,Z_CCMCT_380609B_1.tif,CCMCT,"[[1.1488259535009624, 0.005048975533518764], [...","[-1351.3032792620659, 5259.028690043326]"
2,380609B,A_CCMCT_380609B_1.svs,N1_CCMCT_380609B_1.ndpi,CCMCT,"[[1.1478001380374452, -0.0012548456009253007],...","[2175.912916004514, 4412.461326376817]"
3,518711B,A_CCMCT_518711B_1.svs,N2_CCMCT_518711B_1.ndpi,CCMCT,"[[1.1186036658165646, 0.0023540801321358384], ...","[1354.2968494356485, -464.62880518842064]"
4,518711B,A_CCMCT_518711B_1.svs,Z_CCMCT_518711B_1.tif,CCMCT,"[[1.1490763070570265, 0.015546778329769202], [...","[-3294.561127354221, -1222.1820102596348]"
5,518711B,A_CCMCT_518711B_1.svs,N1_CCMCT_518711B_1.ndpi,CCMCT,"[[1.148300693244396, -0.001957359570592692], [...","[379.1557192435538, 1047.6226786695975]"
6,183715A,A_CCMCT_183715A_1.svs,N2_CCMCT_183715A_1.ndpi,CCMCT,"[[1.1192074084171284, 0.003098871754874583], [...","[456.0092985854998, 38.917894958473795]"
7,183715A,A_CCMCT_183715A_1.svs,Z_CCMCT_183715A_1.tif,CCMCT,"[[1.149332136559863, 0.006747222708883459], [-...","[-2004.539448103431, -2311.81815358037]"
8,183715A,A_CCMCT_183715A_1.svs,N1_CCMCT_183715A_1.ndpi,CCMCT,"[[1.148046889061977, -0.0013068250186180994], ...","[3313.2008770157554, 5846.380113988112]"
9,29609B,A_CCMCT_29609B_1.svs,N2_CCMCT_29609B_1.ndpi,CCMCT,"[[1.1183341050599371, 0.0037280332738240593], ...","[9404.78717895824, -3267.4451938579814]"


In [27]:
qm = quality[quality["target"] == "N1_CCMCT_183715A_1.ndpi"].iloc[0]
qm.b

array([[ 1.14804689, -0.00130683],
       [ 0.00118282,  1.14776979]])

In [28]:
qm.t

array([[3313.20087702],
       [5846.38011399]])